In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
conn_str = os.environ["DB_WAREHOUSE_URI"]
read_sql_query = lambda sql: pd.read_sql_query(sql, conn_str)

In [ ]:
def assert_df_column_value(df, column_name, value):
    assert len(df) == 1
    assert df[column_name].iloc[0] == value
    print(f"OK: {column_name} == {value}")

# Scheduled loads

### All scheduled loads

In [ ]:
query = """
SELECT *
FROM "factScheduledLoads"
"""
read_sql_query(query)

### Q 100: How many loads are scheduled for PO # x?

#### Example: How many loads are scheduled for PO 691306/13121?

In [ ]:
PONumber = "691306/13121"  # Update this to the PONumber you want to query

query = """
SELECT "PONumber", SUM("LoadPieces") AS "LoadPieces", SUM("LoadTons") AS "LoadTons"
FROM "factScheduledLoads"
WHERE "PONumber" = '{PONumber}'
GROUP BY "PONumber"
""".format(PONumber=PONumber)
read_sql_query(query)

### Q 101: How many loads are scheduled for customer code x?

#### Example: How many loads are scheduled for Customer Code 282304?

In [ ]:
CustomerCode = "282326"  # Update this to the PONumber you want to query

query = """
SELECT "CustomerCode", SUM("LoadPieces") AS "LoadPieces", SUM("LoadTons") AS "LoadTons"
FROM "factScheduledLoads"
WHERE "CustomerCode" = '{CustomerCode}'
GROUP BY "CustomerCode"
""".format(CustomerCode=CustomerCode)
read_sql_query(query)

### Q 25: Show the loads scheduled, by referencing pick up number with PO#

In [ ]:
query = """
SELECT "PONumber", "ShippingNumber", "LoadPieces", "LoadTons"
FROM "factScheduledLoads"
"""
read_sql_query(query)

### Q 26: Show the loads scheduled, by customer code

In [ ]:
query = """
SELECT "CustomerCode", SUM("LoadPieces") AS "TotalPieces", SUM("LoadTons") AS "TotalTons"
FROM "factScheduledLoads"
GROUP BY "CustomerCode"
"""
read_sql_query(query)

### Q 94: Show me the PO's with open pick up/shipper #'s

In [ ]:
query = """
SELECT "PONumber", SUM("LoadPieces") AS "TotalPieces", SUM("LoadTons") AS "TotalTons"
FROM "factScheduledLoads"
GROUP BY "PONumber"
"""
read_sql_query(query)

### Q 98: Show me the PO's with open pick up/shipper #'s for these customer codes

In [ ]:
customer_codes = ['282326', '282304']
query = """
SELECT "PONumber", "CustomerCode", SUM("LoadPieces") AS "TotalPieces", SUM("LoadTons") AS "TotalTons"
FROM "factScheduledLoads"
WHERE "CustomerCode" IN ({customer_codes})
GROUP BY "PONumber", "CustomerCode"
""".format(customer_codes=', '.join([f"'{code}'" for code in customer_codes]))
read_sql_query(query)

# Old inventory (non-shipped rolls)

### Q 22: Show the rolls, by customer code, that are 6 months to 1 year

In [ ]:
start = (pd.Timestamp.now() - pd.DateOffset(months=12)).date()
end = (pd.Timestamp.now() - pd.DateOffset(months=6)).date()

query = """
SELECT *
FROM "factInventoryNotShipped"
WHERE "DateEntered" BETWEEN '{start}' AND '{end}'
ORDER BY "DateEntered"
""".format(start=start, end=end)
read_sql_query(query).head(10)

### Q 23: Show the rolls, by customer code, that are 1-2 years old

In [ ]:
start = (pd.Timestamp.now() - pd.DateOffset(months=24)).date()
end = (pd.Timestamp.now() - pd.DateOffset(months=12)).date()

query = """
SELECT *
FROM "factInventoryNotShipped"
WHERE "DateEntered" BETWEEN '{start}' AND '{end}'
ORDER BY "DateEntered"
""".format(start=start, end=end)
read_sql_query(query).head(10)

### Q 24: show the rolls, by customer code, that are 2 years and older

In [ ]:
start = (pd.Timestamp.now() - pd.DateOffset(months=24)).date()

query = """
SELECT *
FROM "factInventoryNotShipped"
WHERE "DateEntered" <= '{start}'
ORDER BY "DateEntered"
""".format(start=start)
read_sql_query(query).head(10)

### Q 102: Inventory by age by customer

In [ ]:
customer = "Trebor Inventory"

query = """
SELECT *
FROM "factInventoryNotShipped"
WHERE "Customer" = '{customer}'
ORDER BY "DateEntered"
""".format(customer=customer)
read_sql_query(query).head(10)

# Purchase order (PO) history

### Q 90: What are the PO's for the last 6 months for these customer codes?

#### Example: What are the PO's for the last 6 months for customer codes 930010, 282304?

In [ ]:
since = (pd.Timestamp.now() - pd.DateOffset(months=6)).date()
customer_codes = ['930010', '282304']

query = """
SELECT "PONumber", "PODate", "CustomerCode", "Customer", "TotalTons" as "OrderedTons", SUM("LoadTons") AS "ProducedTons", SUM("LoadPieces") AS "ProducedPieces"
FROM "factPOProduction"
WHERE "PODate" >= '{since}'
AND "CustomerCode" IN ({customer_codes})
GROUP BY "PONumber", "PODate", "CustomerCode", "Customer", "TotalTons"
""".format(since=since, customer_codes=', '.join([f"'{code}'" for code in customer_codes]))
read_sql_query(query)


### Q 99: Show me the most recent PO for customer code x

#### Example: Show me the most recent PO for customer code 282304

In [ ]:
customer_codes = ['282304']

query = """
SELECT "PONumber", "PODate", "CustomerCode", "Customer", "TotalTons" as "OrderedTons", SUM("LoadTons") AS "ProducedTons", SUM("LoadPieces") AS "ProducedPieces"
FROM "factPOProduction"
WHERE "CustomerCode" IN ({customer_codes})
GROUP BY "PONumber", "PODate", "CustomerCode", "Customer", "TotalTons"
ORDER BY "PODate" DESC
""".format(customer_codes=', '.join([f"'{code}'" for code in customer_codes]))
read_sql_query(query)

### Q 91: Show me the order history for customer X or customer code for the past year.

#### Example: Show me the order history for customer 'American Paper Converting' for the past year.

In [ ]:
customers = ['American Paper Converting']
since = (pd.Timestamp.now() - pd.DateOffset(months=12)).date()

query = """
SELECT "PONumber", "PODate", "CustomerCode", "Customer", "TotalTons" as "OrderedTons", SUM("LoadTons") AS "ProducedTons", SUM("LoadPieces") AS "ProducedPieces"
FROM "factPOProduction"
WHERE "Customer" IN ({customers})
AND "ShippingDate" >= '{since}'
GROUP BY "PONumber", "PODate", "CustomerCode", "Customer", "TotalTons"
ORDER BY "PODate" ASC
""".format(customers=', '.join([f"'{customer}'" for customer in customers]), since=since)
df = read_sql_query(query)

display(df)

df['PODate'] = pd.to_datetime(df['PODate']).dt.date

df.set_index('PODate')[['OrderedTons', 'ProducedTons']].plot(kind='bar', figsize=(28, 5))
plt.title('Ordered vs Produced Tons')
plt.ylabel('Tons')
plt.xticks(rotation=45, ha='right')
plt.show()

#### Example: Show me the order history for customer 'Trebor Inventory' for the past month.

In [ ]:
customers = ['Trebor Inventory']
since = (pd.Timestamp.now() - pd.DateOffset(months=1)).date()

query = """
SELECT "PONumber", "PODate", "CustomerCode", "Customer", "TotalTons" as "OrderedTons", SUM("LoadTons") AS "ProducedTons", SUM("LoadPieces") AS "ProducedPieces"
FROM "factPOProduction"
WHERE "Customer" IN ({customers})
AND "ShippingDate" >= '{since}'
GROUP BY "PONumber", "PODate", "CustomerCode", "Customer", "TotalTons"
ORDER BY "PODate" ASC
""".format(customers=', '.join([f"'{customer}'" for customer in customers]), since=since)
df = read_sql_query(query)

display(df)

df['PODate'] = pd.to_datetime(df['PODate']).dt.date

df.set_index('PODate')[['OrderedTons', 'ProducedTons']].plot(kind='bar', figsize=(28, 5))
plt.title('Ordered vs Produced Tons')
plt.ylabel('Tons')
plt.xticks(rotation=45, ha='right')
plt.show()

### Q 95: Show me the production tons by customer (monthly/annually, specific time frame)

#### Example: Show me the production tons by customer, monthly for last 12 months

In [ ]:
since = (pd.Timestamp.now() - pd.DateOffset(months=12)).date()

query = """
SELECT "Customer", to_char("ShippingDate", 'YYYY-MM') as "ShippingDateYearMonth", SUM("LoadTons") AS "ProducedTons"
FROM "factPOProduction"
WHERE "ShippingDate" >= '{since}'
GROUP BY "Customer", "ShippingDateYearMonth"
ORDER BY "ShippingDateYearMonth" ASC
""".format(since=since)
df = read_sql_query(query)

display(df)

df.pivot(index='ShippingDateYearMonth', columns='Customer', values='ProducedTons').plot(kind='bar', figsize=(28, 5))
plt.title('Produced Tons by Month')
plt.ylabel('Tons')
plt.xticks(rotation=45, ha='right')
plt.show()


#### Example: Show me the production tons by customer, yearly for last 6 years

In [ ]:
since = (pd.Timestamp.now() - pd.DateOffset(months=12 * 6)).date()

query = """
SELECT "Customer", to_char("ShippingDate", 'YYYY') as "ShippingDateYear", SUM("LoadTons") AS "ProducedTons"
FROM "factPOProduction"
WHERE "ShippingDate" >= '{since}'
GROUP BY "Customer", "ShippingDateYear"
ORDER BY "ShippingDateYear" ASC
""".format(since=since)
df = read_sql_query(query)

display(df)

df.pivot(index='ShippingDateYear', columns='Customer', values='ProducedTons').plot(kind='bar', figsize=(28, 5))
plt.title('Produced Tons by Year')
plt.ylabel('Tons')
plt.xticks(rotation=45, ha='right')
plt.show()

### Q 97: Show me the tons shipped for a specific time frame for - all customer, by customer group, or specific customer( PO#, Shipper #, Customer name, Ship Date, Tons Shipped, Grade code, product type)

#### Example: Show me the tons shipped for last month for all customers - group by PO#, Shipper #, Customer name, Ship Date, Grade code, product type)

In [ ]:
since = (pd.Timestamp.now() - pd.DateOffset(months=1)).date()

query = """
SELECT "PONumber", "GradeCode", "Customer", "ShippingDate", "ShippingNumber", SUM("LoadTons") AS "ShippedTons"
FROM "factPOProduction"
WHERE "ShippingDate" >= '{since}'
GROUP BY "PONumber", "GradeCode", "Customer", "ShippingDate", "ShippingNumber"
ORDER BY "ShippingDate" DESC
""".format(since=since)
read_sql_query(query)